In [ ]:
#qiskit version 1.4.3
#qiskit-algorithms version 0.3.1 
#qiskit Aer version 0.17.1
#PySCF version 2.9.0
#qiskit-nature 0.7.2
#numpy version latest
#ase version latest

In [5]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import StatevectorEstimator 

from qiskit_nature.second_q.hamiltonians import LatticeModel
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.hamiltonians.lattices import Lattice, BoundaryCondition
from qiskit.circuit.library import TwoLocal

import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, ActiveSpaceTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock, PUCCD, SUCCD
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA 

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
import numpy as numpy
from qiskit.circuit.library import EfficientSU2, RealAmplitudes 
from qiskit_aer.primitives import Estimator

from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2

from pyscf import gto, scf, cc
from pyscf.pbc import gto
from pyscf.pbc.gto.cell import fromfile

import ase
from ase.io import read
from ase.neighborlist import NeighborList
import time

In [6]:
###
#
###

#1. Use ase to load atomic data (.xyz)

start = time.time()
class DummyUnit:
    def __init__(self, value):
        self.value = value 
        
atoms = read("9007660.xyz")

P = numpy.eye(3, dtype=int)
P = numpy.array([[3,0,0],[0,3,0],[0,0,1]])
supercell = atoms.repeat((1,1,1))
print("Atoms in supercell:", len(supercell))

cell_center = numpy.mean(supercell.get_positions(), axis=0)
radius = 10.0 

positions = supercell.get_positions()
symbols = [a.symbol for a in supercell]

dists = numpy.linalg.norm(positions - cell_center, axis=1)
selected_idx = numpy.where(dists <= radius)[0]

cluster_symbols = [symbols[i] for i in selected_idx]
cluster_positions = positions[selected_idx]

print("Selected atoms count:", len(cluster_symbols))

cluster_atoms = []
for s, pos in zip(cluster_symbols, cluster_positions):
    cluster_atoms.append(f"{s} {pos[0]: .8f} {pos[1]: .8f} {pos[2]: .8f}")

molecule_str = "\n".join(cluster_atoms)
#print("Molecule string (first lines):")


driver = PySCFDriver(atom=molecule_str, unit=DummyUnit("Angstrom"), basis='sto-3g')
freeze_core = FreezeCoreTransformer()
es_problem = driver.run()

es_problem_fc = freeze_core.transform(es_problem) 

active_space = ActiveSpaceTransformer(
    num_electrons=4,#4 
    num_spatial_orbitals=3#3 
    )
es_problem_active = active_space.transform(es_problem_fc)

print("Number of spatial orbitals from driver:", es_problem.num_spatial_orbitals)
print("Number of electrons {total}:", es_problem.num_particles)

#4. Turn into hamiltonian 
second_q_op = es_problem_active.hamiltonian.second_q_op()
#print("Second quantized operator:", second_q_op)

print("Number of frozen spatial orbitals from driver:", es_problem_fc.num_spatial_orbitals)
print("Number of frozen electrons {total}:", es_problem_fc.num_particles)
#5. Map to qubits 
mapper = ParityMapper()
qubit_op = mapper.map(second_q_op) 
num_qubits = qubit_op.num_qubits

print("I mapped to qubits!")

num_spin_orbitals = es_problem_active.num_spatial_orbitals * 2
num_particles = es_problem_active.num_particles

print("num_spin_orbitals:", num_spin_orbitals)
print("num_particles (alpha,beta):", num_particles)


initial_state = HartreeFock(num_spatial_orbitals=es_problem_active.num_spatial_orbitals,
               num_particles=es_problem_active.num_particles,qubit_mapper = mapper) 
#6. Set up and solve with VQE 
#ansatz = SUCCD(es_problem_active.num_spatial_orbitals,
#               es_problem_active.num_particles,
#               mapper,
#               initial_state=initial_state)

ansatz = RealAmplitudes(
    num_qubits=num_qubits,   # number of qubits from mapping
#    rotation_blocks=["ry", "rz"],   # can also try just "ry" or ["rx","ry","rz"]
#    entanglement_blocks="cx",       # CNOT entanglement
    entanglement="full",            # options: "linear", "circular", "full"
    reps=3,                         # number of alternating layers
    insert_barriers=True
) 

#ansatz.initial_state.compose(ansatz)
optimizer = SPSA(maxiter=500) 
estimator = Estimator()

vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
#vqe = VQE(ansatz, optimizer, estimator) <- does not work, causes error in COBYLA object
result = vqe.compute_minimum_eigenvalue(qubit_op)
end = time.time()
elapsed = end - start
#Exact result for comparison
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(qubit_op)

print("VQE ground state energy:", result.eigenvalue.real)
print("Exact ground state energy:", exact_result.eigenvalue.real)
print("Number of qubits:", ansatz.num_qubits)
print(f"VQE calculation took {elapsed:.2f} seconds")

Atoms in supercell: 7
Selected atoms count: 7
Number of spatial orbitals from driver: 81
Number of electrons {total}: (69, 69)
Number of frozen spatial orbitals from driver: 33
Number of frozen electrons {total}: (21, 21)
I mapped to qubits!
num_spin_orbitals: 6
num_particles (alpha,beta): (2, 2)
VQE ground state energy: -1.8593401482297185
Exact ground state energy: -1.9156936897496943
Number of qubits: 6
VQE calculation took 48.13 seconds
